In [ ]:
import sys
import time
import functools
import random
import string
from typing import List, 

In [ ]:
def build_suffix_array_naive(text):
    """
    Construye el arreglo de sufijos generando explícitamente cada sufijo.
    (Esta versión es muy ineficiente en espacio para textos grandes.)
    """
    suffixes = [(text[i:], i) for i in range(len(text))]
    suffixes.sort(key=lambda x: x[0])
    return [pos for (_, pos) in suffixes]

def build_suffix_array(text):
    """
    Construye el arreglo de sufijos de forma mejorada, es decir, ordenando las posiciones
    sin generar explícitamente la cadena del sufijo.
    Se define un comparador que compara carácter a carácter.
    """
    def cmp(i, j):
        n = len(text)
        # Comparar carácter a carácter
        while i < n and j < n:
            if text[i] != text[j]:
                return -1 if text[i] < text[j] else 1
            i += 1
            j += 1
        # Si se alcanza el final de uno, el sufijo más corto es menor
        if i == n and j == n:
            return 0
        return -1 if i == n else 1

In [ ]:
def build_suffix_array(text: str) -> List[int]:
    """
    Construye el arreglo de sufijos de forma eficiente usando el algoritmo de ordenamiento
    basado en comparación de prefijos.
    
    Esta implementación evita la generación explícita de subcadenas durante la comparación,
    lo que la hace mucho más eficiente en memoria para textos grandes.
    """
    def compare_suffixes(i: int, j: int) -> int:
        """Compara dos sufijos que comienzan en las posiciones i y j del texto."""
        n = len(text)
        # Comparar carácter a carácter sin generar subcadenas completas
        while i < n and j < n:
            if text[i] != text[j]:
                return -1 if text[i] < text[j] else 1
            i += 1
            j += 1
        # Si uno de los sufijos llega al final, el más corto es lexicográficamente menor
        return -1 if i == n else 1 if j == n else 0
    
    # Usar sorted() con functools.cmp_to_key para ordenar las posiciones
    # sin generar las subcadenas completas
    positions = list(range(len(text)))
    return sorted(positions, key=functools.cmp_to_key(compare_suffixes))



def find_occurrences(text: str, suffix_array: List[int], query: str) -> List[int]:
    """
    Encuentra todas las ocurrencias de la consulta en el texto usando el arreglo de sufijos.
    
    Retorna una lista de posiciones donde aparece la consulta en el texto.
    Utiliza búsqueda binaria para encontrar el rango de sufijos que comienzan con la consulta.
    """
    if not query:
        return []
    
    n, m = len(text), len(query)
    
    # Función para comparar un sufijo con la consulta
    def compare_with_query(pos: int) -> int:
        """Compara el sufijo que comienza en pos con la consulta."""
        for i in range(m):
            # Si alcanzamos el final del texto, el sufijo es menor que la consulta
            if pos + i >= n:
                return -1
            # Comparar carácter a carácter
            if text[pos + i] != query[i]:
                return -1 if text[pos + i] < query[i] else 1
        # El sufijo comienza con la consulta
        return 0
    
    # Buscar el límite inferior (primer sufijo que comienza con la consulta)
    left, right = 0, len(suffix_array)
    while left < right:
        mid = (left + right) // 2
        suffix_pos = suffix_array[mid]
        if compare_with_query(suffix_pos) < 0:
            left = mid + 1
        else:
            right = mid
    
    # Si no se encontró ningún sufijo que comience con la consulta
    if left == len(suffix_array) or compare_with_query(suffix_array[left]) != 0:
        return []
    
    start = left
    
    # Buscar el límite superior (primer sufijo que no comienza con la consulta)
    left, right = start, len(suffix_array)
    while left < right:
        mid = (left + right) // 2
        suffix_pos = suffix_array[mid]
        if compare_with_query(suffix_pos) == 0:
            left = mid + 1
        else:
            right = mid
    
    end = left
    
    # Retornar todas las posiciones encontradas
    return [suffix_array[i] for i in range(start, end)]



def run_experiments():
    """
    Realiza pruebas con textos sintéticos de diferentes tamaños y número de consultas.
    Imprime una tabla con los tiempos de ejecución.
    """
    # Definir tamaños de texto y cantidad de consultas
    text_sizes = [100_000, 1_000_000, 10_000_000]
    query_counts = [1_000, 10_000, 100_000, 1_000_000]
    
    print("\nEjecutando experimentos de rendimiento:")
    print("=" * 80)
    print(f"{'Tamaño texto':<15} {'Num consultas':<15} {'Tiempo SA (s)':<15} {'Tiempo consultas (s)':<20} {'Tiempo total (s)':<15}")
    print("-" * 80)
    
    for text_size in text_sizes:
        # Generar texto aleatorio (letras minúsculas y espacios)
        print(f"Generando texto aleatorio de {text_size} caracteres...", file=sys.stderr)
        text = ''.join(random.choices(string.ascii_lowercase + ' ', k=text_size))
        
        # Construir el arreglo de sufijos una vez por cada tamaño de texto
        t0 = time.perf_counter()
        suffix_array = build_suffix_array(text)
        t1 = time.perf_counter()
        sa_time = t1 - t0
        
        for qc in query_counts:
            # Generar consultas aleatorias (subcadenas del texto)
            print(f"Generando {qc} consultas aleatorias...", file=sys.stderr)
            queries = []
            for _ in range(qc):
                query_len = random.randint(1, 10)
                start_index = random.randint(0, text_size - query_len - 1)
                query = text[start_index:start_index + query_len]
                queries.append(query)
            
            # Medir tiempo de procesamiento de consultas
            t0_query = time.perf_counter()
            for query in queries:
                _ = find_occurrences(text, suffix_array, query)
            t1_query = time.perf_counter()
            query_time = t1_query - t0_query
            
            total_time = sa_time + query_time
            print(f"{text_size:<15} {qc:<15} {sa_time:<15.3f} {query_time:<20.3f} {total_time:<15.3f}")
    
    print("=" * 80)

# ----------------------------------------
# Ejecución del programa
# ----------------------------------------


In [3]:
run_experiments()


Ejecutando experimentos de rendimiento:
Tamaño texto    Num consultas   Tiempo SA (s)   Tiempo consultas (s) Tiempo total (s)
--------------------------------------------------------------------------------


Generando texto aleatorio de 100000 caracteres...
Generando 1000 consultas aleatorias...
Generando 10000 consultas aleatorias...


100000          1000            0.336           0.021                0.357          
100000          10000           0.336           0.200                0.535          


Generando 100000 consultas aleatorias...


100000          100000          0.336           1.942                2.278          


...

10000000        10000           63.689          28.090               91.779         


Generando 100000 consultas aleatorias...


10000000        100000          63.689          279.458              343.147        


Generando 1000000 consultas aleatorias...


10000000        1000000         63.689          2810.046             2873.735       


In [ ]:
'''
Ejecutando experimentos de rendimiento:
================================================================================
Tamaño texto    Num consultas   Tiempo SA (s)   Tiempo consultas (s) Tiempo total (s)
--------------------------------------------------------------------------------
Generando texto aleatorio de 100000 caracteres...
Generando 1000 consultas aleatorias...
Generando 10000 consultas aleatorias...
100000          1000            0.336           0.021                0.357          
100000          10000           0.336           0.200                0.535          
Generando 100000 consultas aleatorias...
100000          100000          0.336           1.942                2.278          
Generando 1000000 consultas aleatorias...
100000          1000000         0.336           19.429               19.765         
Generando texto aleatorio de 1000000 caracteres...
Generando 1000 consultas aleatorias...
1000000         1000            4.494           0.287                4.781          
Generando 10000 consultas aleatorias...
1000000         10000           4.494           2.568                7.062          
Generando 100000 consultas aleatorias...
1000000         100000          4.494           25.387               29.881         
Generando 1000000 consultas aleatorias...
1000000         1000000         4.494           261.468              265.962        
Generando texto aleatorio de 10000000 caracteres...
Generando 1000 consultas aleatorias...
10000000        1000            63.689          2.723                66.412         
Generando 10000 consultas aleatorias...
10000000        10000           63.689          28.090               91.779         
Generando 100000 consultas aleatorias...
10000000        100000          63.689          279.458              343.147        
Generando 1000000 consultas aleatorias...
10000000        1000000         63.689          2810.046             2873.735       
================================================================================
'''
